In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [ ]:
%cd ../..
!pwd

/Users/ivanosipchyk/dev/investing/ema-crossover
/Users/ivanosipchyk/dev/investing/ema-crossover


In [39]:
import os
import glob
import argparse
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
import gspread
from google.oauth2.service_account import Credentials

from modules.download import *
from modules.indicators import *
from modules.pipeline import *

warnings.simplefilter(action="ignore", category=FutureWarning)

In [11]:
DEFAULT_FILTERS = [
    "Above_EMA_34",
    "Above_EMA_50",
    "Above_EMA_200",
    "MACD_Positive",
    # "MACD_Signal_Negative",
    "Volume",
    "Price",
    "ATR",
]

In [40]:
def write_potential_entries(symbols, sheet_url, sheet_tab="Potential Setups"):
    """
    Append today's potential entry symbols to Google Sheets as one row.

    Args:
        symbols (list of str): List of stock symbols.
        sheet_url (str): Full URL of the Google Sheet.
        sheet_tab (str): Tab name inside the sheet. Default = "Potential Setups".
    """
    if not symbols:
        print("No symbols to write.")
        return
    
    # --- Authenticate ---
    creds_path = os.environ.get("GOOGLE_SHEETS_API_CREDENTIALS")
    if creds_path is None:
        raise RuntimeError("GOOGLE_SHEETS_API_CREDENTIALS not set in environment!")

    creds = Credentials.from_service_account_file(creds_path, scopes=[
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive"
    ])
    client = gspread.authorize(creds)

    # --- Open the sheet & tab ---
    spreadsheet = client.open_by_url(sheet_url)
    worksheet = spreadsheet.worksheet(sheet_tab)

    # --- Format row ---
    today = datetime.now().strftime("%d-%m-%Y")
    symbols_str = ", ".join(symbols)
    row = [today, symbols_str]

    # --- Append row ---
    worksheet.append_row(row, value_input_option="USER_ENTERED")

    print(f"✅ Wrote {symbols_str} to '{sheet_tab}' for {today}.")

def get_previous_trading_day_entries(sheet_url, sheet_tab="Potential Setups"):
    """
    Fetch symbols from the previous trading day row in Google Sheets.

    Args:
        sheet_url (str): Google Sheets URL.
        sheet_tab (str): Tab name. Default = "Potential Setups".

    Returns:
        List[str]: List of symbols from the previous trading day.
    """
    # --- Authenticate ---
    creds_path = os.environ.get("GOOGLE_SHEETS_API_CREDENTIALS")
    if creds_path is None:
        raise RuntimeError("GOOGLE_SHEETS_API_CREDENTIALS not set in environment!")

    creds = Credentials.from_service_account_file(creds_path, scopes=[
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive"
    ])
    client = gspread.authorize(creds)

    # --- Open sheet ---
    spreadsheet = client.open_by_url(sheet_url)
    worksheet = spreadsheet.worksheet(sheet_tab)
    rows = worksheet.get_all_records()  # list of dicts [{"Date": ..., "Symbols": ...}, ...]

    if not rows:
        return []

    # --- Parse dates ---
    for row in rows:
        try:
            row["Date"] = datetime.strptime(row["Date"], "%d-%m-%Y").date()
        except Exception:
            row["Date"] = None

    # --- Find most recent trading day before today ---
    today = datetime.now().date()
    prev_day = today - timedelta(days=1)

    # If weekend, roll back further
    while prev_day.weekday() > 4:  # 5=Saturday, 6=Sunday
        prev_day -= timedelta(days=1)

    # --- Get row for previous trading day ---
    prev_rows = [row for row in rows if row["Date"] == prev_day]
    if not prev_rows:
        print(f"No entries found for {prev_day}.")
        return []

    symbols_str = prev_rows[-1]["Symbols"]  # take last entry for that day
    symbols = [s.strip() for s in symbols_str.split(",") if s.strip()]

    return symbols

In [ ]:
def tp_distribution(max_tp, skip_tp, coef):
    """
    Geometric distribution of TP sizes.
    """
    weights = np.array([coef**i for i in range(max_tp-skip_tp)])
    weights = weights / weights.sum()
    weights = [0.0 for i in range(skip_tp)] + weights.tolist()
    return weights

def generate_trades(
    stock_data_for_entries, 
    stop_rule: str = 'crossover', 
    sl_offset_pc: float = 0.0, 
    max_tp: int = 5,
    skip_tp: int = 1,
    tp_coef: float = 2.0,
    position_size: float = 100.0
) -> pd.DataFrame:
    """
    Generate trades from stock data with entry, SL, TP levels, TP sizes.

    Args:
        stock_data_for_entries (dict): Mapping symbol -> DataFrame with OHLC and indicators.
        stop_rule (str): Rule for stop loss placement ("crossover" or "previous").
        sl_offset_pc (float): Stop loss offset in % of risk.
        max_tp (int): Number of take profit levels.
        skip_tp (int): Number of TPs to skip at start.
        tp_coef (float): Coefficient for geometric TP size distribution.
        position_size (float): Default position size.

    Returns:
        pd.DataFrame: Trades with entry, stop loss, BE, risk, TPs and TP sizes.
    """
    today_trades = []

    for sym, df in stock_data_for_entries.items():
        if len(df) < 3:
            continue

        pre_cross_day = df.iloc[-3]
        cross_day = df.iloc[-2]
        today = df.iloc[-1]

        # --- Entry price ---
        if today["High"] >= cross_day["High"]:
            entry_price = max(cross_day["High"], today["Open"])
        else:
            continue

        # --- Stop loss ---
        if stop_rule == "crossover":
            stop_loss = cross_day["Low"]
        elif stop_rule == "previous":
            stop_loss = pre_cross_day["Low"]
        else:
            raise ValueError("Invalid stop rule")

        # Apply SL offset
        stop_loss -= sl_offset_pc / 100 * (entry_price - stop_loss)

        # Number of shares
        share_n = position_size / entry_price

        # Filter invalid trades
        if stop_loss >= entry_price:
            continue

        # --- Risk & targets ---
        risk_per_share = entry_price - stop_loss
        total_risk = risk_per_share * position_size

        tp_levels = [entry_price + (i + 1) * risk_per_share for i in range(max_tp)]
        be_price = entry_price  # currently entry price, can be adjusted later

        # --- TP size distribution ---
        tp_sizes = tp_distribution(max_tp, skip_tp=skip_tp, coef=tp_coef)
        tp_sizes = [round(s, 2) for s in tp_sizes]

        # --- Save trade ---
        trade = {
            "Date": datetime.now().date(),
            "Symbol": df["Symbol"].values[0], 
            "EntryPrice": round(entry_price, 2),
            "ShareN": round(share_n, 2),
            "PositionSize": position_size,
            "StopLoss": round(stop_loss, 2),
            "Risk": round(total_risk, 2),
            "BEPrice": round(be_price, 2),
            "Stop Rule": stop_rule, 
            "Offset": sl_offset_pc, 
            "Max TP": max_tp,
            "Skip TP": skip_tp,
        }

        for i, (tp_level, tp_size) in enumerate(zip(tp_levels, tp_sizes), start=1):
            trade[f"TP_{i}"] = round(tp_level, 2)
            trade[f"TP_{i}_Size"] = tp_size

        today_trades.append(trade)

    # --- Create DataFrame ---
    df_trades = pd.DataFrame(today_trades)

    # Reorder columns
    base_cols = [
        "Date", "Symbol", "EntryPrice", "ShareN", "PositionSize", "StopLoss", "Risk", "BEPrice",
        "Stop Rule", "Offset", "Max TP", "Skip TP",
    ]
    tp_cols = [col for col in df_trades.columns if col.startswith("TP_")]
    df_trades = df_trades[base_cols + tp_cols]

    return df_trades

In [85]:
import os
from datetime import datetime
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd

def write_trades_to_sheet(
    trades_df: pd.DataFrame,
    sheet_url: str,
    stop_rule: str,
    sl_offset_pc: float,
    max_tp: int,
    skip_tp: int,
    tp_coef: float,
    position_size: float = 100.0
) -> str:
    """
    Append trade DataFrame to Google Sheets. Creates a new tab if needed.
    
    Args:
        trades_df (pd.DataFrame): DataFrame of trades.
        sheet_url (str): Google Sheets URL.
        stop_rule (str): Stop rule used.
        sl_offset_pc (float): Stop loss offset percentage.
        max_tp (int): Maximum number of TPs.
        skip_tp (int): Number of initial TPs to skip.
        tp_coef (float): Coefficient for TP distribution.
        position_size (float): Size of position per trade.
    
    Returns:
        str: Name of the sheet tab where trades were written.
    """
    if trades_df.empty:
        print("No trades to write.")
        return ""
    
    # --- Authenticate ---
    creds_path = os.environ.get("GOOGLE_SHEETS_API_CREDENTIALS")
    if not creds_path:
        raise RuntimeError("GOOGLE_SHEETS_API_CREDENTIALS not set in environment!")

    creds = Credentials.from_service_account_file(
        creds_path, 
        scopes=["https://www.googleapis.com/auth/spreadsheets",
                "https://www.googleapis.com/auth/drive"]
    )
    client = gspread.authorize(creds)

    # --- Open spreadsheet ---
    spreadsheet = client.open_by_url(sheet_url)

    # --- Determine tab name based on parameters ---
    tab_name = f"Trades_{stop_rule}_SL{sl_offset_pc}_TP{max_tp}_skip{skip_tp}_coef{tp_coef}"

    # --- Create tab if it doesn't exist ---
    try:
        worksheet = spreadsheet.worksheet(tab_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=tab_name, rows=1000, cols=50)
        print(f"Created new tab: {tab_name}")

    # --- Convert Date column to string ---
    df_to_write = trades_df.copy()
    if "Date" in df_to_write.columns:
        df_to_write["Date"] = df_to_write["Date"].astype(str)

    # --- Prepare values including header ---
    all_values = [df_to_write.columns.tolist()] + df_to_write.values.tolist()

    # --- Append all at once ---
    existing_rows = worksheet.get_all_values()
    start_row = len(existing_rows) + 1  # append after last row

    worksheet.update(
        f"A{start_row}",
        all_values,
        value_input_option="USER_ENTERED"
    )

    print(f"✅ Appended {len(trades_df)} trades to tab '{tab_name}' with headers")
    return tab_name

In [ ]:
from datetime import datetime
import os
import pandas as pd

def main(filters, 
         symbols_list=None, 
         sheet_url="https://docs.google.com/spreadsheets/d/16HZfFIu37ZG7kRgLDI9SqS5xPhb3pn3iY2ubOymVseU/edit#gid=1171524967",
         potential_tab="Potential Setups"):
    """
    Full daily pipeline:
    - Download and label stock data
    - Find potential entries
    - Write today's potential entries to Google Sheets
    - Fetch previous trading day's entries
    - Generate trades for previous day entries
    - Append trades to Google Sheets

    Args:
        filters (list): List of filters to apply for potential entries
        symbols_list (list, optional): List of symbols. Defaults to hardcoded test list.
        sheet_url (str): Google Sheets URL
        potential_tab (str): Tab name for potential entries

    Returns:
        dict: {"today": list of today's potential entries,
               "previous_day": list of previous trading day entries,
               "trades_df": DataFrame of trades}
    """
    print("⬇️ Downloading fresh data...")
    shift = 0

    # Default test symbols if none provided
    if symbols_list is None:
        symbols_list = ['XYZ', 'STT', 'POR', 'FOX', 'EVRG', 'MMYT', 'BHF', 'MGNI', 'ATRC']

    # --- Download OHLCV ---
    stock_data = download_data(
        symbols=symbols_list,
        period="500d",
        interval="1d",
        batch_size=100,
    )

    # --- Label data ---
    stock_data_labeled = apply_to_dict(stock_data, process_symbol_df)

    # --- Run potential entries ---
    potential_entries = find_potential_entries(
        stock_data_labeled,
        shift=shift,
        filters=filters,
    )

    print("\n📈 Potential entries:")
    print(potential_entries)

    # --- Save potential entries to Google Sheets ---
    if potential_entries:
        write_potential_entries(
            symbols=potential_entries,
            sheet_url=sheet_url,
            sheet_tab=potential_tab
        )

    # --- Get previous trading day's entries ---
    prev_day_entries = get_previous_trading_day_entries(
        sheet_url=sheet_url,
        sheet_tab=potential_tab
    )

    print("\n📅 Previous trading day entries:")
    print(prev_day_entries)

    # --- Filter stock data for entries ---
    stock_data_for_entries = {sym: df for sym, df in stock_data_labeled.items() if sym in prev_day_entries}

    # --- Generate trades DataFrame ---
    trades_df = generate_trades(
        stock_data_for_entries,
        stop_rule="crossover",
        sl_offset_pc=0,
        max_tp=5,
        skip_tp=0
    )

    # --- Write trades to Google Sheets ---
    if not trades_df.empty:
        write_trades_to_sheet(
            trades_df=trades_df,
            sheet_url=sheet_url,
            stop_rule="crossover",
            sl_offset_pc=0,
            max_tp=5,
            skip_tp=0,
            tp_coef=1.0,
            position_size=100
        )

    # --- Return everything ---
    return {
        "today": potential_entries,
        "previous_day": prev_day_entries,
        "trades_df": trades_df
    }

In [87]:
results = main(DEFAULT_FILTERS)

⬇️ Downloading fresh data...


Processing symbols:   0%|          | 0/9 [00:00<?, ?it/s]


📈 Potential entries:
[]

📅 Previous trading day entries:
['XYZ', 'STT', 'POR', 'FOX', 'EVRG', 'MMYT', 'BHF', 'MGNI', 'ATRC']
Created new tab: Trades_crossover_SL0_TP5_skip0_coef1.0


/var/folders/q7/0s4jjvb94hb_2_90j2545plw0000gn/T/ipykernel_29269/1146075931.py:74: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(


✅ Appended 5 trades to tab 'Trades_crossover_SL0_TP5_skip0_coef1.0' with headers
